In [26]:
gpu=0

In [72]:
import sys
sys.path.append('../')
%matplotlib inline
from config import domainData
from config import num_classes as NUM_CLASSES
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import torchvision
import utils
from torchvision import datasets, transforms
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from alexnet import alexnet

In [28]:
net = alexnet
net.load_state_dict(torch.load('alexnet.pth'))
# print(net)

In [29]:
net = nn.Sequential(*list(net.children())[:-2])
print(net)

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (1): ReLU(inplace)
  (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (5): ReLU(inplace)
  (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1)
  (7): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace)
  (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (11): ReLU(inplace)
  (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (13): ReLU(inplace)
  (14): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (15): Lambda(
  )
  (16): Sequential(
    (0): Lambda(
    )
    (1): Linear(in_features=9216, out_features=4096, bias

In [30]:
src = domainData['amazon']
tar = domainData['webcam']

In [31]:
src_transforms = transforms.Compose([
    transforms.Resize(256),
#     transforms.CenterCrop(227),
#     transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(227, scale=(0.50,1.0), ratio=(1.,1.)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[1.,1.,1.]),
    transforms.Lambda(lambda x: torch.stack([x[2],x[1],x[0]])), # RGB -> BGR
    transforms.Lambda(lambda x: x * 255.)
])
tar_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
#     transforms.RandomResizedCrop(224, scale=(0.25,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[1.,1.,1.]),
    transforms.Lambda(lambda x: torch.stack([x[2],x[1],x[0]])),
    transforms.Lambda(lambda x: x * 255.)
])

In [32]:
src_dataset = datasets.ImageFolder(src, transform=src_transforms)
tar_dataset = datasets.ImageFolder(tar, transform=tar_transforms)
srcSampler = torch.utils.data.sampler.RandomSampler(src_dataset)
tarSampler = torch.utils.data.sampler.RandomSampler(tar_dataset)
srcDataLen = len(src_dataset)
tarDataLen = len(tar_dataset)
use_gpu = True and torch.cuda.is_available()

In [33]:
opt = {
    'src': 'Amazon',
    'tar': 'Webcam',
    'manual_seed':1,
    'batchSize':80,
    'use_gpu': use_gpu,
    'num_classes': 31,
    'epochs': 75,
    'momentum': 0.9,
    'lr': 5e-4,
    'lr_sch': 0,
    'lr_sch_gamma': 0.1,
    'p_lr_decay': 75,
    'n0': 1.,
    'alpha': 10,
    'beta': 0.75,
    'betas': (0.5,0.99),
    'net_wtDcy': 1e-5,
    'btl_wtDcy': 1e-5,
    'srcDataLen': srcDataLen,
    'tarDataLen': tarDataLen
}

In [34]:
torch.manual_seed(opt['manual_seed'])
if opt['use_gpu']: torch.cuda.manual_seed(opt['manual_seed'])

In [35]:
print("use_gpu: ", opt['use_gpu'])

use_gpu:  True


In [36]:
src_dataloader = torch.utils.data.DataLoader(src_dataset, batch_size=opt['batchSize'], 
                                             shuffle=(srcSampler is None), sampler=srcSampler,
                                            num_workers=2, pin_memory=True, drop_last=False)
tar_dataloader = torch.utils.data.DataLoader(tar_dataset, batch_size=opt['batchSize'],
                                            shuffle=(tarSampler is None), sampler=tarSampler,
                                            num_workers=2, pin_memory=True, drop_last=False)

In [37]:
def init_weights(m):
#     print(m)
    if isinstance(m, nn.Linear):
        init.xavier_normal(m.weight)
        init.constant(m.bias, 0.1)

In [38]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(nn.Linear(4096,256),
                                       nn.ReLU(inplace=True), nn.Linear(256, opt['num_classes']))
        self.classifier.apply(init_weights)
    
    def forward(self, x):
        x = self.classifier(x)
        return x

In [39]:
net2 = Model()

In [40]:
if opt['use_gpu']:
    net = net.cuda()
    net2 = net2.cuda()
    torch.backends.cudnn.enabled=True
    torch.backends.cudnn.benchmark=True

To computer norm of various parameters

```python
for name, param in net2.named_parameters():
    nrm = torch.norm(param, 2)
    zero = param.eq(0.).float().sum()
    nele = torch.numel(param)
    print(name, nrm.data[0], nele, zero.data[0])
```

In [41]:
criterion = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=1)

In [42]:
netfeatwt, netfeatbias = list(), list()
for name, param in net.named_parameters():
    if name in ['10.weight', '12.weight', '16.1.weight', '19.1.weight']: netfeatwt.append(param)
    elif name in ['10.bias', '12.bias', '16.1.bias', '19.1.bias']: netfeatbias.append(param)
    else: param.requires_grad=False

In [43]:
for name, param in net.named_parameters():
    print(name, param.requires_grad)

0.weight False
0.bias False
4.weight False
4.bias False
8.weight False
8.bias False
10.weight True
10.bias True
12.weight True
12.bias True
16.1.weight True
16.1.bias True
19.1.weight True
19.1.bias True


In [44]:
net2_weight, net2_bias = list(), list()
for name, param in net2.named_parameters():
    if 'weight' in name: net2_weight.append(param)
    elif 'bias' in name: net2_bias.append(param)

In [45]:
sgd_params = [
    {'params': netfeatwt, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['net_wtDcy'], 'lr_mul': 1., 'name': 'netfeatwt'},
    {'params': netfeatbias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['net_wtDcy'], 'lr_mul': 2., 'name': 'netfeatbias'},
    {'params': net2_weight, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['btl_wtDcy'], 'lr_mul': 10., 'name': 'net2wt'},
    {'params': net2_bias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['btl_wtDcy'], 'lr_mul': 20., 'name': 'net2bias'}
]

# optimizer = optim.Adam(net2.parameters(), lr=opt['lr'], betas=opt['betas'], weight_decay=opt['weight_decay'])
# optimizer = optim.Adam(sgd_params, betas=opt['betas'])

optimizer_2 = optim.SGD(sgd_params)
# optimizer_2 = optim.SGD(net2.parameters(), lr=opt['lr'], momentum=opt['momentum'], weight_decay=opt['btl_wtDcy'])

lr_sch = None
if opt['lr_sch']=='step': lr_sch = optim.lr_scheduler.StepLR(optimizer, 20, opt['lr_sch_gamma'])
if opt['lr_sch']=='exponential': lr_sch = optim.lr_scheduler.ExponentialLR(optimizer_2, opt['lr_sch_gamma'])

In [46]:
def get_validation_acc(cm=False):
    tarData = iter(tar_dataloader)
    totalCorrects = 0.
    if cm: y_preds, y_true = list(), list()
    for tarimgs, tarlbls in tarData:
        tarimgs = tarimgs.cuda() if opt['use_gpu'] else tarimgs
        tarlbls = tarlbls.cuda() if opt['use_gpu'] else tarlbls      
        tarimgs = Variable(tarimgs, volatile=True)

        feat_ = net(tarimgs)
        logits = net2(feat_)

        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, tarlbls).float().sum()
        if cm: y_preds.append(preds.cpu().numpy()), y_true.append(tarlbls.cpu().numpy())
    valAcc = totalCorrects / opt['tarDataLen']
    if cm: return valAcc, y_preds, y_true
    return valAcc

In [47]:
srcClsLoss_plt, srcAcc_plt, valAcc_plt = list(), list(), list()

In [48]:
p = np.linspace(float(1./opt['p_lr_decay']),1,opt['p_lr_decay'])

for epoch in range(opt['epochs']):
    srcData = iter(src_dataloader)
    totalCorrects = 0.
    totalClsLoss = 0.
#     experiment.log_current_epoch(epoch)
    
    n_p = opt['n0'] / pow((1. + opt['alpha'] * p[epoch]), (opt['beta']))
    print("n_p: ", n_p)
    for param_group in optimizer_2.param_groups:
        param_group['lr'] = opt['lr'] * param_group['lr_mul'] * n_p
        
    for srcimgs, srclbls in srcData:
        srcimgs = srcimgs.cuda() if opt['use_gpu'] else srcimgs
        srclbls = srclbls.cuda() if opt['use_gpu'] else srclbls
        srcimgs, srclbls = Variable(srcimgs), Variable(srclbls)
        
        feat_ = net(srcimgs)
        logits = net2(feat_)
        
        clsloss = criterion(logits, srclbls)
        totalClsLoss += clsloss.data[0] * opt['batchSize']
        
        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, srclbls.data).float().sum()
               
        optimizer_2.zero_grad()
        clsloss.backward()
        optimizer_2.step()
        
        if lr_sch: lr_sch.step()
        
    srcAcc = totalCorrects / opt['srcDataLen']
    srcLoss = totalClsLoss / opt['srcDataLen']
    valAcc = get_validation_acc()
#     experiment.log_metric("src_clsLoss", srcLoss, step=epoch)
#     experiment.log_metric("src_clsAcc", srcAcc, step=epoch)
#     experiment.log_metric('val_accuracy', valAcc, step=epoch)
#     experiment.log_epoch_end(epoch)
    print("Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, validation acc: {:.4f}".
          format(epoch+1, srcLoss, srcAcc, valAcc))
    srcClsLoss_plt.append(srcLoss), srcAcc_plt.append(srcAcc), valAcc_plt.append(valAcc)

n_p:  0.910398960683
Epoch: 1, train loss: 2.4647, train acc: 0.4178, validation acc: 0.4780
n_p:  0.837535537332
Epoch: 2, train loss: 1.1056, train acc: 0.7061, validation acc: 0.4591
n_p:  0.776969504241
Epoch: 3, train loss: 0.8686, train acc: 0.7604, validation acc: 0.5044
n_p:  0.725725807496
Epoch: 4, train loss: 0.7305, train acc: 0.8005, validation acc: 0.5245
n_p:  0.68173161988
Epoch: 5, train loss: 0.6397, train acc: 0.8165, validation acc: 0.5094
n_p:  0.643495658493
Epoch: 6, train loss: 0.5402, train acc: 0.8474, validation acc: 0.5057
n_p:  0.609915886892
Epoch: 7, train loss: 0.5016, train acc: 0.8562, validation acc: 0.5208
n_p:  0.580159187126
Epoch: 8, train loss: 0.4622, train acc: 0.8658, validation acc: 0.5258
n_p:  0.55358331165
Epoch: 9, train loss: 0.3737, train acc: 0.8843, validation acc: 0.5484
n_p:  0.529684678724
Epoch: 10, train loss: 0.3626, train acc: 0.8910, validation acc: 0.5220
n_p:  0.508062512101
Epoch: 11, train loss: 0.3347, train acc: 0.8988, 

```
valAcc, y_preds, y_true = get_validation_acc(cm=True)

y_preds = [d for sublist in y_preds for d in sublist]
y_true = [d for sublist in y_true for d in sublist]

cm = confusion_matrix(y_true, y_preds)

np.set_printoptions(precision=2)
plt.figure(figsize=(20,20))
utils.plot_confusion_matrix(cm, classes=src_dataset.classes, normalize=False)
plt.savefig('cm.png')
plt.close()
```

In [70]:
plt.figure(figsize=(20,10))
plt.title('Src and Tar accuracy')
# plt.ylim([0,1])
plt.plot(srcAcc_plt, label='srcAcc')
plt.plot(valAcc_plt, label='tarAcc')
plt.legend(loc=(0.80,0.10), scatterpoints=1)
plt.savefig('acc1.png')
plt.close()

In [85]:
with open('report.html', 'w+') as file:
    file.write(
        '''<!DOCTYPE html>
        <html>
        <style>
        img {
            float: right;
        }
        </style>
        <body>
        <h4>HyperParams</h4>
        <table><img src="./acc1.png" alt="Accuracy plots" style="margin-left:15px;">
        <tr>
            <th>Param</th><th>Value</th>
        </tr>'''
    )
    for k,v in opt.items():
        file.write('<tr><td>%s </td><td> %s</td></tr>' % (k,v))
    file.write('</table>\n')
    file.write('<h4>Classifier Network</h4>')
    file.write(str(net2)+'\n')
    file.write('<h4>Base Network - Alexnet</h4>')
    file.write(str(net)+'\n')
    file.write('<h4>Layers frozen</h4>\n')
    for n,param in net.named_parameters():
        file.write('<span style="font-weight:bold">%s</span> %s\n' % (n, param.requires_grad))
    file.write('</body></html>')

In [82]:
with open('experiment.log', 'w+') as file:
    file.write('Experiment log\nEpoch\tClsLoss\tSrcAcc\tTarAcc\n')
    epoch = 1.
    for l,s,t in zip(srcClsLoss_plt, srcAcc_plt, valAcc_plt):
        file.write("Epoch:{}\t{:.4f}\t{:.4f}\t{:.4f}\n".format(epoch,l,s,t))
        epoch = epoch + 1.